In [6]:
import pandas as pd 
import noaa_ftp


In [ ]:
class DailyFile:
    
    cols_list = ['ID','YEAR','MONTH','ELEMENT']
    widths_list = [11,4,2,4]
    
    def __init__(self, station_id, data_folder= 'data/'):
        self.ftp = noaa_ftp.NoaaFTP('pub/data/ghcn/daily/all')
        print(f'Retrieving {station_id}.dly')
        self.ftp.retrieve_file(f'{station_id}.dly', data_folder)
        self.ftp.quit()
        
        for n in range(1,32):
            # append column names to colsList
            self.cols_list.append(f'VALUE{n}')
            self.cols_list.append(f'MFLAG{n}')
            self.cols_list.append(f'QFLAG{n}')
            self.cols_list.append(f'SFLAG{n}')
        
            # append column widths to widthsList
            self.widths_list.append(5)
            self.widths_list.append(1)
            self.widths_list.append(1)
            self.widths_list.append(1)
            
        # build dataframe
        self.df_raw = pd.read_fwf(
                            f'{data_folder}{station_id}.dly'
                            ,header=None
                            ,widths=self.widths_list
                            ,names=self.cols_list
                            ,index_col=False)

        self.df = pd.DataFrame(columns=['ID','YEAR','MONTH','ELEMENT','VALUE','DAY'])
        
        for i in range(1,32):
            df_append = self.df_raw.filter(['ID','YEAR','MONTH','ELEMENT',f'VALUE{i}'])
            df_append.rename(columns={f'VALUE{i}':'VALUE'},inplace=True)
            df_append['DAY'] = i
            self.df = self.df.append(df_append)

        # 1. change ID column name to StationID
        self.df = self.df.rename(columns={'ID': 'StationID'})

        # 2. change datatype of value to double
        self.df['VALUE'] = self.df['VALUE'].astype('float')

        # 3. filter out -9999 values
        self.df = self.df[ self.df['VALUE']>-9999 ]
        
    def mm_to_inches(self, row):
        """
        converts the VALUE from millimeters to inches
        because murica is "special"
        """
        return row['VALUE'] / 25.4
    
    def c_to_f(self, row):
        # (0°C × 9/5) + 32 = 32°F
        return ( ((((row['VALUE']/10) * 9) / 5) + 32))
    
    def winter_season(self, row):
        """
        formats the YEAR field to be like the winter of 90-91, 03-04
        these years in this format go from June to May
        """
        if row['MONTH'] < 6:
            return str(row['YEAR'] - 1)[2:4] + '-' + str(row['YEAR'])[2:4]
        else:
            return str(row['YEAR'])[2:4] + '-' + str(row['YEAR'] + 1)[2:4]
        
    